In [1]:
from typing import Callable, Optional, Tuple
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, ListedColormap, to_rgba

# ---------- Readers & transforms ----------
def _read_first_two_maps(path: str):
    """Return list of the first up to two 2D maps from a CASTEP .f25 file."""
    float_re = re.compile(r'[-+]?(?:\d+\.\d*|\.\d+|\d+)[eEdD][-\+]?\d+')
    maps = []
    with open(path, 'r', errors='ignore') as fh:
        for line in fh:
            m = re.search(r'MAPN\s+(\d+)\s+(\d+)', line)
            if not m: 
                continue
            nx, ny = int(m.group(1)), int(m.group(2))
            next(fh); next(fh)  # skip origin & lattice lines
            data = []
            while len(data) < nx * ny:
                nums = float_re.findall(next(fh))
                if nums:
                    data.extend(float(s.replace('D','E').replace('d','E')) for s in nums)
            maps.append(np.array(data, float).reshape(ny, nx))
            if len(maps) == 2:
                break
    return maps

def _default_transform(arr: np.ndarray) -> np.ndarray:
    # Match the orientation of your example image
    arr = np.flip(arr, axis=1)
    arr = np.rot90(arr, k=3)
    arr = np.flip(arr, axis=0)
    arr = np.flip(arr, axis=1)
    return arr

# ---------- Colormap tuned to the reference look ----------
def make_soft_plasma(bg_hex: str = "#a9a3d7", low_frac: float = 0.45, gamma: float = 0.5) -> ListedColormap:
    """
    Start from 'plasma' but replace the darkest end with a soft lavender background.
    - bg_hex   : low-end lavender tone
    - low_frac : fraction of the lower range to soften
    - gamma    : shape of the blend ramp (<=1 gives more pastel background)
    """
    base = plt.colormaps["plasma"](np.linspace(0, 1, 256))
    bg = np.array(to_rgba(bg_hex))
    n = int(256 * low_frac)
    ramp = np.linspace(0.0, 1.0, n) ** gamma       # 0→1
    base[:n, :] = (1 - ramp)[:, None] * bg + ramp[:, None] * base[:n, :]
    return ListedColormap(base)

# ---------- Main plotter ----------
def plot_charge_density_f25(
    file_path: str,
    out_path: Optional[str] = None,
    *,
    apply_transform: bool = True,
    transform_fn: Optional[Callable[[np.ndarray], np.ndarray]] = None,
    charge_percentiles: Tuple[float, float] = (5.0, 80.0),
    n_levels_charge: int = 320,
    dpi: int = 900,
    figsize: Tuple[float, float] = (3.5, 3.5),
    show_colorbar: bool = True,
    hide_colorbar_ticks: bool = True,
    cmap: Optional[ListedColormap] = None
):
    # Read the first (charge) map
    maps = _read_first_two_maps(file_path)
    if not maps:
        raise ValueError("No maps found in the .f25 file.")
    total = maps[0]

    if apply_transform:
        tf = transform_fn or _default_transform
        total = tf(total)

    # LogNorm on positive values with percentile clipping
    pos = total[total > 0]
    if pos.size == 0:
        raise ValueError("Charge map contains no positive values required for LogNorm.")
    p_lo, p_hi = charge_percentiles
    vmin_total = float(np.percentile(pos, p_lo))
    vmax_total = float(np.percentile(pos, p_hi))
    min_pos = float(np.min(pos))
    if vmin_total <= 0: vmin_total = max(min_pos, 1e-12 * vmax_total)
    if vmin_total >= vmax_total: vmin_total = max(min_pos, 1e-3 * vmax_total)

    total_clip = total.copy()
    total_clip[total_clip < vmin_total] = vmin_total
    total_clip[total_clip > vmax_total] = vmax_total

    if cmap is None:
        cmap = make_soft_plasma(bg_hex="#131315", low_frac=0.7, gamma=0.9)

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    levels = np.geomspace(vmin_total, vmax_total, n_levels_charge)
    cf = ax.contourf(
        total_clip, levels=levels, cmap=cmap,
        norm=LogNorm(vmin=vmin_total, vmax=vmax_total), antialiased=True
    )

    if show_colorbar:
        cbar = fig.colorbar(cf, ax=ax, pad=0.02, shrink=0.92)
        if hide_colorbar_ticks:
            cbar.set_ticks([])      # no numbers
            cbar.set_label("")      # no title
            cbar.ax.tick_params(length=0)
        cbar.outline.set_linewidth(0.6)

    # Clean, paper-ready axes
    ax.set_xlabel(""); ax.set_ylabel(""); ax.set_title("")
    ax.set_xticks([]); ax.set_yticks([])
    ax.tick_params(which="both", bottom=False, top=False, left=False, right=False,
                   labelbottom=False, labelleft=False)
    ax.set_aspect("equal", adjustable="box")
    ax.grid(False)
    plt.tight_layout()

    if out_path:
        plt.savefig(out_path, dpi=1000, bbox_inches="tight", transparent=True)
    return fig, ax

# fig, ax = plot_charge_density_f25(
#     "/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/GaO_PRISTINE_18HF_ECHG.f25",
#     out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/GaO_PRISTINE_18HF_ECHG.png",
#     show_colorbar=False,
#     hide_colorbar_ticks=True,
#     charge_percentiles=(0.001, 98.5),
#     n_levels_charge=150, dpi=1000, figsize=(3.5, 3.5)
# )


In [2]:
"""
Charge–spin overlay for CASTEP .f25 maps (paper‑ready).

Charge density:
    colormap='plasma' with LogNorm scaled to the (p_lo, p_hi) percentiles of
    the strictly positive values; values are clipped to [vmin, vmax] to avoid holes.

Spin density:
    colormap='seismic' (red=+spin, blue=–spin).
    Default normalization is TwoSlopeNorm centered at 0 with limits ±|spin| at
    the given percentile. Alternatives:
        spin_norm_mode='minmax'  → TwoSlopeNorm with true min/max.
        spin_norm_mode='symlog'  → SymLogNorm with true range.

API:
    plot_charge_spin_overlay(
        file_path, out_path=None, apply_transform=True, transform_fn=None,
        charge_percentiles=(1.0, 99.5), spin_percentile=99.5,
        spin_norm_mode='quantile', symlog_linthresh_frac=0.02,
        spin_alpha=0.60, dpi=900, figsize=(3.5, 3.5),
        charge_cmap='plasma', spin_cmap='seismic',
        n_levels_charge=320, n_levels_spin=320,
        show_colorbar=True
    ) -> (fig, ax)
"""
from typing import Callable, Optional, Tuple
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, TwoSlopeNorm, SymLogNorm, ListedColormap, to_rgba
from matplotlib.colorbar import ColorbarBase
from mpl_toolkits.axes_grid1 import make_axes_locatable


# --------------------------- I/O & transforms ---------------------------

def _read_maps(path: str) -> Tuple[np.ndarray, np.ndarray]:
    float_re = re.compile(r'[-+]?(?:\d+\.\d*|\.\d+|\d+)[eEdD][-\+]?\d+')
    maps = []
    with open(path, 'r', errors='ignore') as fh:
        for line in fh:
            if line.startswith('-%-1MAPN'):
                nx, ny = map(int, line.split()[1:3])
                next(fh); next(fh)
                data = []
                while len(data) < nx * ny:
                    nums = float_re.findall(next(fh))
                    if nums:
                        data.extend(float(s.replace('D', 'E').replace('d', 'E')) for s in nums)
                maps.append(np.array(data).reshape(ny, nx))
                if len(maps) == 2:
                    break
    if len(maps) < 2:
        raise ValueError("Expected at least two maps (total, spin) in the .f25 file.")
    return maps[0], maps[1]


def _default_transform(arr: np.ndarray) -> np.ndarray:
    arr = np.flip(arr, axis=1)
    arr = np.rot90(arr, k=3)
    arr = np.flip(arr, axis=0)
    arr = np.flip(arr, axis=1)
    return arr


# --------------------------- Colormaps ----------------------------------

def make_soft_plasma(bg_hex: str = "#a9a3d7", low_frac: float = 0.45, gamma: float = 0.85) -> ListedColormap:
    """
    Start from 'plasma' but replace the darkest end with a soft lavender background.
    - bg_hex   : low-end lavender tone
    - low_frac : fraction of the lower range to soften
    - gamma    : shape of the blend ramp (<=1 gives more pastel background)
    """
    base = plt.colormaps["plasma"](np.linspace(0, 1, 256))
    bg = np.array(to_rgba(bg_hex))
    n = int(256 * low_frac)
    ramp = np.linspace(0.0, 1.0, n) ** gamma       # 0→1
    base[:n, :] = (1 - ramp)[:, None] * bg + ramp[:, None] * base[:n, :]
    return ListedColormap(base)


def make_transparent_center_cmap(
    base_cmap="seismic",
    *,
    center_frac: float = 0.12,
    alpha_gamma: float = 3.0,
    center_cut_frac: float = 0.08,
) -> ListedColormap:
    """
    Build a diverging colormap with:
      - alpha=0 around the center (±center_frac/2), ramping to 1 with exponent gamma,
      - reduced 'white' at the center by compressing the center of the base cmap.

    center_frac     total fractional width of the transparent band around 0
    alpha_gamma     >1 sharpens the transition (thinner pale band)
    center_cut_frac compresses the *colors* near the center so the first visible colors are
                    more saturated (reduces white rim). 0–0.12 is typical.
    """
    base = plt.colormaps[base_cmap]
    n = 256
    t = np.linspace(0, 1, n)
    cut = float(center_cut_frac) / 2.0
    # Compress central whites in the base cmap
    def remap(x):
        if x < 0.5:
            return (x / 0.5) * (0.5 - cut)
        else:
            return 1.0 - ((1.0 - x) / 0.5) * (0.5 - cut)
    tt = np.array([remap(x) for x in t])
    rgba = base(tt)

    # Alpha ramp around center
    mid = n // 2
    half = max(1, int(center_frac * n / 2.0))
    dist = np.abs(np.arange(n) - mid) / half          # 0 at center, 1 at band edge
    alpha = np.clip(dist, 0.0, 1.0) ** float(alpha_gamma)
    rgba[:, 3] = alpha
    return ListedColormap(rgba)


def opaque_copy(cmap) -> ListedColormap:
    """Return an opaque copy of a cmap (alpha=1 everywhere)."""
    arr = plt.colormaps[cmap](np.linspace(0, 1, 256)) if isinstance(cmap, str) else cmap(np.linspace(0, 1, 256))
    arr[:, 3] = 1.0
    return ListedColormap(arr)


# --------------------------- Main plot ----------------------------------

def plot_charge_spin_overlay(
    file_path: str,
    out_path: Optional[str] = None,
    *,
    apply_transform: bool = True,
    transform_fn: Optional[Callable[[np.ndarray], np.ndarray]] = None,
    charge_percentiles: Tuple[float, float] = (1.0, 99.5),
    spin_percentile: float = 99.5,
    spin_norm_mode: str = "quantile",
    symlog_linthresh_frac: float = 0.02,
    spin_alpha: float = 0.60,
    dpi: int = 900,
    figsize: Tuple[float, float] = (3.5, 3.5),
    charge_cmap: str = "plasma",
    spin_cmap: str = "seismic",
    n_levels_charge: int = 320,
    n_levels_spin: int = 320,
    show_colorbar: bool = True,
):
    # --- read & transform ---
    total, spin = _read_maps(file_path)
    if apply_transform:
        tf = transform_fn or _default_transform
        total = tf(total)
        spin = tf(spin)

    # --- charge scaling & clipping (LogNorm, avoid holes) ---
    pos = total[total > 0]
    if pos.size == 0:
        raise ValueError("Charge map contains no positive values required for LogNorm.")
    p_lo, p_hi = charge_percentiles
    vmin_total = float(np.percentile(pos, p_lo))
    vmax_total = float(np.percentile(pos, p_hi))
    min_pos = float(np.min(pos))
    if vmin_total <= 0:
        vmin_total = max(min_pos, 1e-12 * vmax_total)
    if vmin_total >= vmax_total:
        vmin_total = max(min_pos, 1e-3 * vmax_total)

    total_clip = total.copy()
    total_clip[total_clip < vmin_total] = vmin_total
    total_clip[total_clip > vmax_total] = vmax_total

    # --- spin normalization ---
    if spin_norm_mode == "quantile":
        L = float(np.percentile(np.abs(spin), spin_percentile))
        L = L if L > 0 else float(np.max(np.abs(spin)))
        norm_spin = TwoSlopeNorm(vmin=-L, vcenter=0.0, vmax=L)
        spin_plot = np.clip(spin, -L, L)
    elif spin_norm_mode == "minmax":
        L = float(np.max(np.abs(spin)))
        norm_spin = TwoSlopeNorm(vmin=-L, vcenter=0.0, vmax=L)
        spin_plot = spin
    elif spin_norm_mode == "symlog":
        L = float(np.max(np.abs(spin)))
        linthresh = max(L * symlog_linthresh_frac, np.finfo(float).eps)
        norm_spin = SymLogNorm(linthresh=linthresh, vmin=-L, vmax=L)
        spin_plot = spin
    else:
        raise ValueError("spin_norm_mode must be 'quantile', 'minmax', or 'symlog'.")

    # --- make near‑zero spin fully transparent (and thin the pale rim) ---
    # Use a slightly wider band by default to suppress white edges.
    if spin_norm_mode == "symlog":
        thresh = max(linthresh, np.finfo(float).eps)
    else:
        thresh_rel = 0.06                       # 6% of range; increase to widen transparent band
        thresh = max(1e-14, thresh_rel * L)

    spin_plot = np.ma.masked_where(np.abs(spin_plot) < thresh, spin_plot)

    # --- set up figure ---
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)

    # --- charge layer ---
    charge_levels = np.geomspace(vmin_total, vmax_total, n_levels_charge)
    charge_cmap_eff = make_soft_plasma(bg_hex="#131315", low_frac=0.45, gamma=0.85)
    ax.contourf(
        total_clip,
        levels=charge_levels,
        cmap=charge_cmap_eff,
        norm=LogNorm(vmin=vmin_total, vmax=vmax_total),
        antialiased=True,
    )

    # --- spin layer: transparent-center cmap and levels that skip 0 ---
    center_frac = min(0.5, 2.0 * max(thresh / L, 1e-8))
    spin_cmap_eff = make_transparent_center_cmap(
        base_cmap=spin_cmap,
        center_frac=center_frac,
        alpha_gamma=3.0,          # steeper alpha ramp → less white halo
        center_cut_frac=0.08,     # compress center colors → first visible colors more saturated
    )

    n_neg = n_levels_spin // 2
    n_pos = n_levels_spin - n_neg
    neg = np.linspace(-L, -thresh, n_neg, endpoint=True)
    pos = np.linspace( thresh,  L, n_pos, endpoint=True)
    spin_levels = np.r_[neg, pos]

    cf_spin = ax.contourf(
        spin_plot,                 # masked array
        levels=spin_levels,
        cmap=spin_cmap_eff,
        norm=norm_spin,
        antialiased=False,         # avoids light halos
        alpha=spin_alpha,
        extend="both",
    )

    # --- colorbar: same height as the plot, no numbers, no label ---
    if show_colorbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="4%", pad=0.02)

        # Use an opaque cmap for the colorbar so it doesn't show a "hole" in the center.
        cbar = ColorbarBase(cax, cmap=opaque_copy(spin_cmap), norm=norm_spin, orientation="vertical")

        # Remove ticks and label for a clean, paper‑style colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])
        # Optional: keep or hide the outline; comment the next line to keep it.
        # cbar.outline.set_visible(False)

    # --- cosmetics: no axes numbers/titles ---
    ax.set_xlabel(""); ax.set_ylabel(""); ax.set_title("")
    ax.set_xticks([]); ax.set_yticks([])
    ax.tick_params(which="both",
                   bottom=False, top=False, left=False, right=False,
                   labelbottom=False, labelleft=False)
    ax.set_aspect('equal', adjustable='box')
    ax.grid(False)

    # Tight layout without shrinking the colorbar
    plt.tight_layout(pad=0.0)

    if out_path:
        plt.savefig(out_path, dpi=dpi, bbox_inches='tight', transparent=True)
    return fig, ax


# --------------------------- CLI example --------------------------------

# if __name__ == "__main__":
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_1x2x2_OCTA_OCTA_VACS_SCF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_1x2x2_OCTA_OCTA_VACS_SCF_ECHG.png",
#         charge_percentiles=(0.01, 99.5),
#         spin_percentile=97,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_1x2x2_TETRA_TETRA_VACS_SCF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_1x2x2_TETRA_TETRA_VACS_SCF_ECHG.png",
#         charge_percentiles=(0.01, 99.0),
#         spin_percentile=97,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_1x2x2_OCTA_VAC_SCF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_1x2x2_OCTA_VAC_SCF_ECHG.png",
#         charge_percentiles=(0.01, 99.0),
#         spin_percentile=97.5,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_1x2x2_TETRA_VAC_SCF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_1x2x2_TETRA_VAC_SCF_ECHG.png",
#         charge_percentiles=(0.01, 99.0),
#         spin_percentile=98,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_2x2x2_VAC_TETRA_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_2x2x2_VAC_TETRA_ECHG.png",
#         charge_percentiles=(0.01, 99.2),
#         spin_percentile=99.4,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_2x2x2_VAC_OCTAHEDRAL_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_2x2x2_VAC_OCTAHEDRAL_ECHG.png",
#         charge_percentiles=(0.01, 99.2),
#         spin_percentile=99.4,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )
#     fig, ax = plot_charge_spin_overlay(
#         file_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/Ga2O3_Ovac_1_SCF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/Ga2O3_Ovac_1_SCF_ECHG.png",
#         charge_percentiles=(0.01, 99.0),
#         spin_percentile=100,
#         spin_norm_mode="quantile",
#         show_colorbar=False,
#         dpi=900,
#         figsize=(3.5, 3.5),
#         spin_alpha=0.85,
#     )

#     plt.show()


In [3]:
from typing import Callable, Optional, Tuple, Sequence
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, ListedColormap, to_rgba


# ---------- Readers & transforms ----------

def _read_first_two_maps(path: str) -> Sequence[np.ndarray]:
    """Return list of the first up to two 2D maps from a CASTEP .f25 file."""
    float_re = re.compile(r'[-+]?(?:\d+\.\d*|\.\d+|\d+)[eEdD][-\+]?\d+')
    maps = []
    with open(path, 'r', errors='ignore') as fh:
        for line in fh:
            m = re.search(r'MAPN\s+(\d+)\s+(\d+)', line)
            if not m:
                continue
            nx, ny = int(m.group(1)), int(m.group(2))
            # skip origin & lattice lines
            next(fh)
            next(fh)
            data = []
            while len(data) < nx * ny:
                nums = float_re.findall(next(fh))
                if nums:
                    data.extend(
                        float(s.replace("D", "E").replace("d", "E"))
                        for s in nums
                    )
            maps.append(np.array(data, float).reshape(ny, nx))
            if len(maps) == 2:
                break
    return maps


def _default_transform(arr: np.ndarray) -> np.ndarray:
    """Match the orientation you tuned in the previous figures."""
    arr = np.flip(arr, axis=1)
    arr = np.rot90(arr, k=3)
    arr = np.flip(arr, axis=0)
    arr = np.flip(arr, axis=1)
    return arr


# ---------- Colormap tuned to vacancy / spin‑overlay style ----------

def make_soft_plasma(
    bg_hex: str = "#131315",
    low_frac: float = 0.45,
    gamma: float = 0.85,
) -> ListedColormap:
    """
    Start from 'plasma' but replace the darkest end with a soft background.

    bg_hex   : low-end background tone
    low_frac : fraction of the lower range to soften
    gamma    : shape of the blend ramp (<=1 gives more pastel background)
    """
    base = plt.colormaps["plasma"](np.linspace(0, 1, 256))
    bg = np.array(to_rgba(bg_hex))
    n = int(256 * low_frac)
    ramp = np.linspace(0.0, 1.0, n) ** gamma   # 0→1
    base[:n, :] = (1 - ramp)[:, None] * bg + ramp[:, None] * base[:n, :]
    return ListedColormap(base)


# Optional helper if you want *one* charge scale for many systems
def compute_global_charge_limits(
    file_paths: Sequence[str],
    *,
    apply_transform: bool = True,
    transform_fn: Optional[Callable[[np.ndarray], np.ndarray]] = None,
    percentiles: Tuple[float, float] = (0.01, 99.2),
) -> Tuple[float, float]:
    """
    Scan several .f25 files and return a common (vmin, vmax) for LogNorm.

    Use this once, then pass charge_limits=(vmin, vmax) into
    plot_charge_density_f25 *and* your vacancy overlay function.
    """
    all_pos = []
    for path in file_paths:
        maps = _read_first_two_maps(path)
        if not maps:
            continue
        total = maps[0]
        if apply_transform:
            tf = transform_fn or _default_transform
            total = tf(total)
        pos = total[total > 0]
        if pos.size:
            all_pos.append(pos.ravel())

    if not all_pos:
        raise ValueError("No positive charge values found in any file.")

    all_pos = np.concatenate(all_pos)
    p_lo, p_hi = percentiles
    vmin = float(np.percentile(all_pos, p_lo))
    vmax = float(np.percentile(all_pos, p_hi))
    return vmin, vmax


# ---------- Main plotter for pristine case ----------

def plot_charge_density_f25(
    file_path: str,
    out_path: Optional[str] = None,
    *,
    apply_transform: bool = True,
    transform_fn: Optional[Callable[[np.ndarray], np.ndarray]] = None,
    # Used only if charge_limits is None:
    charge_percentiles: Tuple[float, float] = (0.01, 99.2),
    # If given, overrides percentiles and enforces a global scale:
    charge_limits: Optional[Tuple[float, float]] = None,
    n_levels_charge: int = 320,
    dpi: int = 900,
    figsize: Tuple[float, float] = (3.5, 3.5),
    show_colorbar: bool = True,
    hide_colorbar_ticks: bool = True,
    cmap: Optional[ListedColormap] = None,
):
    """
    Plot total charge density from a CASTEP .f25 file (pristine system).

    - If charge_limits is None → vmin/vmax from charge_percentiles.
    - If charge_limits=(vmin, vmax) → uses those directly (for consistency
      with vacancy plots).
    """
    # Read the first (charge) map
    maps = _read_first_two_maps(file_path)
    if not maps:
        raise ValueError("No maps found in the .f25 file.")
    total = maps[0]

    if apply_transform:
        tf = transform_fn or _default_transform
        total = tf(total)

    # LogNorm on positive values with percentile clipping
    pos = total[total > 0]
    if pos.size == 0:
        raise ValueError("Charge map contains no positive values required for LogNorm.")

    # --- choose vmin/vmax ---
    if charge_limits is None:
        p_lo, p_hi = charge_percentiles
        vmin_total = float(np.percentile(pos, p_lo))
        vmax_total = float(np.percentile(pos, p_hi))
    else:
        vmin_total, vmax_total = map(float, charge_limits)

    min_pos = float(np.min(pos))
    if vmin_total <= 0:
        vmin_total = max(min_pos, 1e-12 * vmax_total)
    if vmin_total >= vmax_total:
        vmin_total = max(min_pos, 1e-3 * vmax_total)

    # clip to avoid "holes" with LogNorm
    total_clip = total.copy()
    total_clip[total_clip < vmin_total] = vmin_total
    total_clip[total_clip > vmax_total] = vmax_total

    # shared soft-plasma colormap (same as vacancy overlay)
    if cmap is None:
        cmap = make_soft_plasma()

    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    levels = np.geomspace(vmin_total, vmax_total, n_levels_charge)

    cf = ax.contourf(
        total_clip,
        levels=levels,
        cmap=cmap,
        norm=LogNorm(vmin=vmin_total, vmax=vmax_total),
        antialiased=True,
    )

    # Colorbar (optional, usually hidden for paper)
    if show_colorbar:
        cbar = fig.colorbar(cf, ax=ax, pad=0.02, shrink=0.92)
        if hide_colorbar_ticks:
            cbar.set_ticks([])      # no numbers
            cbar.set_label("")      # no title
            cbar.ax.tick_params(length=0)
        cbar.outline.set_linewidth(0.6)

    # Clean, paper-ready axes
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_title("")
    ax.set_xticks([])
    ax.set_yticks([])
    ax.tick_params(
        which="both",
        bottom=False, top=False, left=False, right=False,
        labelbottom=False, labelleft=False,
    )
    ax.set_aspect("equal", adjustable="box")
    ax.grid(False)
    plt.tight_layout()

    if out_path:
        plt.savefig(out_path, dpi=1000, bbox_inches="tight", transparent=True)

    return fig, ax


# ---------------------- Example usage ----------------------
# if __name__ == "__main__":
#     # Option 1: let the pristine plot pick its own percentiles
#     fig, ax = plot_charge_density_f25(
#         "/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/ECHG/GaO_PRISTINE_18HF_ECHG.f25",
#         out_path="/Users/felipe/FelipePhysicsPhD/Ga2O3Paper/FormalPaper/ECHGFigures/GaO_PRISTINE_18HF_ECHG.png",
#         show_colorbar=False,
#         hide_colorbar_ticks=True,
#         charge_percentiles=(0.01, 99.2),
#         n_levels_charge=320,
#         dpi=1000,
#         figsize=(3.5, 3.5),
#     )

    # Option 2 (recommended for consistency):
    # compute_global_charge_limits(...) *once* over all pristine+vacancy files,
    # then call plot_charge_density_f25 with charge_limits=(vmin, vmax)
    # and do the same in your vacancy overlay function.
